Обнаружение и сегментация объектов

Антонов Михаил Евгеньевич, М-26

## Часть 1: Метрика IoU и алгоритм NMS

### 1.1. Intersection over Union (IoU)

**Определение:** IoU (Intersection over Union) - метрика для оценки пересечения двух bounding box'ов. Вычисляется как отношение площади пересечения к площади объединения.

In [ ]:
from typing import List, Tuple, Dict, Optional
from io import BytesIO

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import requests
import torch
import torchvision
from torchvision import transforms as T
from torchvision.models.detection import fasterrcnn_resnet50_fpn, maskrcnn_resnet50_fpn
from torchvision.models.detection import FasterRCNN_ResNet50_FPN_Weights, MaskRCNN_ResNet50_FPN_Weights
from torchvision.models.segmentation import deeplabv3_resnet101
from torchvision.models.segmentation import DeepLabV3_ResNet101_Weights
from PIL import Image

# Константы
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
DEFAULT_CONFIDENCE_THRESHOLD = 0.5
DEFAULT_IOU_THRESHOLD = 0.5

# Названия классов COCO
COCO_CATEGORIES = [
    '__background__', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
    'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign',
    'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
    'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
    'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat',
    'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass',
    'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange',
    'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch',
    'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
    'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator',
    'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush'
]


def calculate_iou(box1: List[float], box2: List[float]) -> float:
    """Вычисляет IoU (Intersection over Union) для двух bounding box.
    
    Args:
        box1: Первый box в формате [x1, y1, x2, y2]
        box2: Второй box в формате [x1, y1, x2, y2]
        
    Returns:
        Значение IoU от 0 до 1
    """
    x1_1, y1_1, x2_1, y2_1 = box1
    x1_2, y1_2, x2_2, y2_2 = box2

    # Координаты пересечения
    x_left = max(x1_1, x1_2)
    y_top = max(y1_1, y1_2)
    x_right = min(x2_1, x2_2)
    y_bottom = min(y2_1, y2_2)

    # Проверка на отсутствие пересечения
    if x_right < x_left or y_bottom < y_top:
        return 0.0

    intersection = (x_right - x_left) * (y_bottom - y_top)
    area1 = (x2_1 - x1_1) * (y2_1 - y1_1)
    area2 = (x2_2 - x1_2) * (y2_2 - y1_2)
    union = area1 + area2 - intersection

    return intersection / union if union > 0 else 0.0


def non_max_suppression(
    boxes: np.ndarray,
    scores: np.ndarray,
    iou_threshold: float = DEFAULT_IOU_THRESHOLD
) -> List[int]:
    """Применяет Non-Maximum Suppression к набору bounding boxes.
    
    Args:
        boxes: Массив boxes формы (N, 4) в формате [x1, y1, x2, y2]
        scores: Массив confidence scores формы (N,)
        iou_threshold: Порог IoU для подавления
        
    Returns:
        Список индексов оставшихся boxes
    """
    boxes = np.array(boxes)
    scores = np.array(scores)

    x1, y1, x2, y2 = boxes[:, 0], boxes[:, 1], boxes[:, 2], boxes[:, 3]
    areas = (x2 - x1) * (y2 - y1)
    order = scores.argsort()[::-1]

    keep = []
    while order.size > 0:
        i = order[0]
        keep.append(int(i))

        # Вычисление IoU с оставшимися boxes
        xx1 = np.maximum(x1[i], x1[order[1:]])
        yy1 = np.maximum(y1[i], y1[order[1:]])
        xx2 = np.minimum(x2[i], x2[order[1:]])
        yy2 = np.minimum(y2[i], y2[order[1:]])

        w = np.maximum(0.0, xx2 - xx1)
        h = np.maximum(0.0, yy2 - yy1)
        intersection = w * h
        union = areas[i] + areas[order[1:]] - intersection
        iou = intersection / union

        # Оставляем только boxes с IoU ниже порога
        inds = np.where(iou <= iou_threshold)[0]
        order = order[inds + 1]

    return keep


# Демонстрация IoU
print("Тестирование функции IoU:")
print("=" * 40)

test_cases = [
    ([10, 10, 50, 50], [10, 10, 50, 50], "Полное перекрытие"),
    ([10, 10, 50, 50], [20, 20, 60, 60], "Частичное перекрытие"),
    ([10, 10, 50, 50], [70, 70, 100, 100], "Нет перекрытия"),
]

for box1, box2, description in test_cases:
    iou = calculate_iou(box1, box2)
    print(f"{description}: IoU = {iou:.3f}")

# Визуализация IoU
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
colors = ['red', 'blue']

for idx, (box1, box2, title) in enumerate(test_cases):
    ax = axes[idx]
    iou = calculate_iou(box1, box2)
    
    for i, box in enumerate([box1, box2]):
        x1, y1, x2, y2 = box
        rect = patches.Rectangle(
            (x1, y1), x2 - x1, y2 - y1,
            linewidth=2, edgecolor=colors[i], facecolor='none',
            alpha=0.7, label=f'Box {i+1}'
        )
        ax.add_patch(rect)

    ax.set_xlim(0, 110)
    ax.set_ylim(0, 110)
    ax.set_title(f"{title}\nIoU = {iou:.3f}")
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.grid(True, alpha=0.3)
    ax.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Демонстрация NMS
print("Тестирование Non-Maximum Suppression:")
print("=" * 50)

test_boxes = [
    [20, 20, 80, 80],
    [30, 30, 90, 90],
    [25, 25, 85, 85],
    [100, 100, 150, 150],
    [110, 110, 160, 160],
    [200, 200, 250, 250],
]
test_scores = [0.9, 0.8, 0.7, 0.85, 0.75, 0.95]

print("Исходные bounding boxes:")
for i, (box, score) in enumerate(zip(test_boxes, test_scores)):
    print(f"  Box {i+1}: {box}, score={score:.2f}")

# Тестирование с разными порогами
for threshold in [0.3, 0.5, 0.7]:
    selected = non_max_suppression(test_boxes, test_scores, iou_threshold=threshold)
    print(f"\nNMS (IoU threshold={threshold}):")
    print(f"  Оставшиеся boxes: {[i+1 for i in selected]}")
    for idx in selected:
        print(f"    Box {idx+1}: {test_boxes[idx]}, score={test_scores[idx]:.2f}")

In [ ]:
# Визуализация NMS на примере перекрывающихся прямоугольников
demo_boxes = [
    [30, 30, 100, 100],
    [50, 50, 120, 120],
    [40, 40, 110, 110],
    [70, 70, 140, 140],
]
demo_scores = [0.85, 0.92, 0.78, 0.88]

print("Демонстрация NMS на перекрывающихся прямоугольниках:")
print("-" * 50)

# Расчёт IoU между всеми парами
print("\nМатрица IoU:")
for i in range(len(demo_boxes)):
    for j in range(i+1, len(demo_boxes)):
        iou = calculate_iou(demo_boxes[i], demo_boxes[j])
        print(f"  IoU(Box{i+1}, Box{j+1}) = {iou:.3f}")

# Применение NMS
selected = non_max_suppression(demo_boxes, demo_scores, iou_threshold=0.5)
print(f"\nПосле NMS (threshold=0.5): оставлен Box {selected[0]+1} (score={demo_scores[selected[0]]:.2f})")

# Визуализация
fig, axes = plt.subplots(1, 2, figsize=(12, 6))
colors = ['red', 'blue', 'green', 'orange']

for ax_idx, (title, show_selected) in enumerate([
    ("Исходные прямоугольники", False),
    ("После NMS (threshold=0.5)", True)
]):
    ax = axes[ax_idx]
    
    for i, (box, score) in enumerate(zip(demo_boxes, demo_scores)):
        x1, y1, x2, y2 = box
        
        if show_selected:
            if i in selected:
                rect = patches.Rectangle(
                    (x1, y1), x2-x1, y2-y1,
                    linewidth=3, edgecolor=colors[i], facecolor=colors[i],
                    alpha=0.5, label=f'Box {i+1}: {score:.2f}'
                )
            else:
                rect = patches.Rectangle(
                    (x1, y1), x2-x1, y2-y1,
                    linewidth=1, edgecolor='gray', facecolor='gray',
                    alpha=0.2, linestyle='--'
                )
        else:
            rect = patches.Rectangle(
                (x1, y1), x2-x1, y2-y1,
                linewidth=2, edgecolor=colors[i], facecolor=colors[i],
                alpha=0.4, label=f'Box {i+1}: {score:.2f}'
            )
        
        ax.add_patch(rect)
        
        # Номер в центре
        cx, cy = (x1+x2)/2 - 5, (y1+y2)/2
        if not show_selected or i in selected:
            ax.text(cx, cy, str(i+1), fontsize=12, fontweight='bold', ha='center')

    ax.set_xlim(0, 200)
    ax.set_ylim(0, 200)
    ax.set_title(title, fontsize=14)
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.grid(True, alpha=0.3)
    ax.legend(loc='upper left', fontsize=10)

plt.tight_layout()
plt.show()

print("\nВывод: NMS оставляет только Box с максимальным score среди перекрывающихся")

######2 Обнаружение объектов с помощью предобученной модели.


In [ ]:
print("Загрузка модели Faster R-CNN...")
print(f"Устройство: {DEVICE}")

# Загрузка с современным API (weights вместо pretrained)
model = fasterrcnn_resnet50_fpn(weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT)
model = model.to(DEVICE)
model.eval()

print(f"Модель загружена успешно")
print(f"Количество классов: {model.roi_heads.box_predictor.cls_score.out_features}")

In [ ]:
def load_image_from_url(url: str, timeout: int = 10) -> Optional[Image.Image]:
    """Загружает изображение по URL.
    
    Args:
        url: URL изображения
        timeout: Таймаут запроса в секундах
        
    Returns:
        PIL Image или None при ошибке
    """
    try:
        response = requests.get(url, timeout=timeout)
        response.raise_for_status()
        return Image.open(BytesIO(response.content)).convert("RGB")
    except Exception as e:
        print(f"Ошибка загрузки {url}: {e}")
        return None


def transform_image(image: Image.Image) -> torch.Tensor:
    """Преобразует PIL Image в тензор для модели."""
    transform = T.Compose([T.ToTensor()])
    return transform(image).unsqueeze(0).to(DEVICE)


# URL тестовых изображений COCO
IMAGE_URLS = [
    "http://images.cocodataset.org/val2017/000000039769.jpg",
    "http://images.cocodataset.org/val2017/000000000139.jpg",
    "http://images.cocodataset.org/val2017/000000037777.jpg",
]

# Альтернативные URL на случай недоступности
FALLBACK_URLS = [
    "http://images.cocodataset.org/val2017/000000000285.jpg",
    "http://images.cocodataset.org/val2017/000000000724.jpg",
]

print("Загрузка тестовых изображений...")
images = []
transformed_images = []

for i, url in enumerate(IMAGE_URLS):
    img = load_image_from_url(url)
    if img is None and i < len(FALLBACK_URLS):
        print(f"Пробуем альтернативный URL...")
        img = load_image_from_url(FALLBACK_URLS[i])
    
    if img is not None:
        images.append(img)
        transformed_images.append(transform_image(img))
        print(f"Изображение {len(images)} загружено: {img.size}")

print(f"\nЗагружено {len(images)} изображений")

# Детекция объектов
print("\nВыполнение детекции...")
all_predictions = []

with torch.no_grad():
    for i, img_tensor in enumerate(transformed_images):
        prediction = model(img_tensor)
        all_predictions.append(prediction[0])
        n_objects = len(prediction[0]['boxes'])
        print(f"Изображение {i+1}: найдено {n_objects} потенциальных объектов")

In [ ]:
def visualize_detections(
    image: Image.Image,
    predictions: Dict,
    threshold: float = DEFAULT_CONFIDENCE_THRESHOLD,
    title: str = "Результаты детекции"
) -> Tuple[plt.Figure, List[str]]:
    """Визуализирует результаты детекции объектов.
    
    Args:
        image: Исходное изображение
        predictions: Словарь с boxes, labels, scores
        threshold: Порог уверенности
        title: Заголовок графика
        
    Returns:
        Кортеж (figure, список обнаруженных классов)
    """
    keep = predictions['scores'] > threshold
    boxes = predictions['boxes'][keep].cpu().numpy()
    labels = predictions['labels'][keep].cpu().numpy()
    scores = predictions['scores'][keep].cpu().numpy()

    fig, ax = plt.subplots(1, figsize=(14, 10))
    ax.imshow(image)

    cmap = plt.cm.get_cmap('tab20', len(COCO_CATEGORIES))
    detected_objects = []

    for box, label, score in zip(boxes, labels, scores):
        if label >= len(COCO_CATEGORIES):
            continue

        class_name = COCO_CATEGORIES[label]
        color = cmap(label)

        x1, y1, x2, y2 = box
        rect = patches.Rectangle(
            (x1, y1), x2 - x1, y2 - y1,
            linewidth=2, edgecolor=color, facecolor='none'
        )
        ax.add_patch(rect)

        text = f"{class_name}: {score:.2f}"
        ax.text(
            x1, y1 - 8, text,
            bbox=dict(boxstyle="round,pad=0.3", facecolor=color, alpha=0.8),
            fontsize=9, color='white', fontweight='bold'
        )
        detected_objects.append(class_name)

    ax.axis('off')
    unique = len(set(detected_objects))
    ax.set_title(
        f"{title}\nОбъектов: {len(boxes)} | Уникальных классов: {unique} | Порог: {threshold}",
        fontsize=14, fontweight='bold', pad=20
    )

    plt.tight_layout()
    return fig, detected_objects


# Визуализация результатов
print("Визуализация результатов детекции:")
print("=" * 70)

for i, (img, pred) in enumerate(zip(images, all_predictions)):
    fig, detected = visualize_detections(img, pred, threshold=0.5, title=f"Изображение {i+1}")
    plt.show()

    keep = pred['scores'] > 0.5
    scores = pred['scores'][keep].cpu().numpy()
    
    print(f"\nИзображение {i+1}:")
    print(f"  Размер: {img.size}")
    print(f"  Обнаружено объектов: {len(detected)}")
    print(f"  Уникальных классов: {len(set(detected))}")
    if len(scores) > 0:
        print(f"  Уверенность: min={scores.min():.3f}, max={scores.max():.3f}, mean={scores.mean():.3f}")
    print("-" * 70)

In [ ]:
def analyze_threshold_effects(
    image: Image.Image,
    predictions: Dict,
    image_num: int,
    thresholds: List[float] = [0.9, 0.7, 0.5, 0.3]
) -> Dict:
    """Анализирует влияние порога уверенности на результаты детекции.
    
    Args:
        image: Исходное изображение
        predictions: Результаты детекции
        image_num: Номер изображения
        thresholds: Список порогов для анализа
        
    Returns:
        Словарь со статистикой по каждому порогу
    """
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    axes = axes.ravel()
    
    cmap = plt.cm.get_cmap('tab20', len(COCO_CATEGORIES))
    stats = {}

    for idx, threshold in enumerate(thresholds):
        ax = axes[idx]
        
        keep = predictions['scores'] > threshold
        boxes = predictions['boxes'][keep].cpu().numpy()
        labels = predictions['labels'][keep].cpu().numpy()
        scores = predictions['scores'][keep].cpu().numpy()

        stats[threshold] = {
            'count': len(boxes),
            'scores': scores,
            'labels': labels
        }

        ax.imshow(image)

        for box, label, score in zip(boxes, labels, scores):
            if label >= len(COCO_CATEGORIES):
                continue

            color = cmap(label)
            x1, y1, x2, y2 = box
            rect = patches.Rectangle(
                (x1, y1), x2-x1, y2-y1,
                linewidth=1.5, edgecolor=color, facecolor='none'
            )
            ax.add_patch(rect)

            if threshold <= 0.5:
                class_name = COCO_CATEGORIES[label]
                ax.text(
                    x1, y1-6, f"{class_name}: {score:.2f}",
                    bbox=dict(boxstyle="round,pad=0.2", facecolor=color, alpha=0.7),
                    fontsize=7, color='white'
                )

        ax.axis('off')
        unique = len(set([COCO_CATEGORIES[l] for l in labels if l < len(COCO_CATEGORIES)]))
        ax.set_title(f"Порог: {threshold}\nОбъектов: {len(boxes)} | Классов: {unique}", fontsize=11)

    plt.suptitle(f"Изображение {image_num}: Влияние порога уверенности", fontsize=16, y=1.02)
    plt.tight_layout()
    plt.show()

    return stats


# Анализ влияния порога
print("Исследование влияния порога уверенности:")
print("=" * 80)

all_stats = []
for i, (img, pred) in enumerate(zip(images, all_predictions)):
    print(f"\nАнализ изображения {i+1}")
    stats = analyze_threshold_effects(img, pred, i+1)
    all_stats.append(stats)

    print(f"  Статистика по порогам:")
    for threshold in [0.9, 0.7, 0.5, 0.3]:
        count = stats[threshold]['count']
        if count > 0:
            scores = stats[threshold]['scores']
            print(f"    Порог {threshold}: {count} объектов, уверенность {scores.min():.3f}-{scores.max():.3f}")
        else:
            print(f"    Порог {threshold}: нет объектов")

In [ ]:
# Сводный анализ
print("\n" + "=" * 80)
print("СВОДНЫЙ АНАЛИЗ ВЛИЯНИЯ ПОРОГА УВЕРЕННОСТИ")
print("=" * 80)

for img_num, stats in enumerate(all_stats, 1):
    print(f"\nИзображение {img_num}:")
    
    counts = [stats[t]['count'] for t in [0.9, 0.7, 0.5, 0.3]]
    print(f"  Количество объектов: 0.9→{counts[0]} | 0.7→{counts[1]} | 0.5→{counts[2]} | 0.3→{counts[3]}")

    if counts[0] > 0:
        increase = ((counts[2] - counts[0]) / counts[0]) * 100
        print(f"  Прирост объектов (0.9→0.5): +{increase:.1f}%")

    if stats[0.5]['count'] > 0:
        low_conf = sum(1 for s in stats[0.5]['scores'] if s < 0.7)
        if low_conf > 0:
            print(f"  Объекты с уверенностью <0.7 при пороге 0.5: {low_conf}")

In [ ]:
# Детальный анализ первого изображения
if len(all_stats) > 0:
    print("\nДетальный анализ первого изображения:")
    print("=" * 60)

    img1_stats = all_stats[0]
    t90 = img1_stats[0.9]
    t50 = img1_stats[0.5]

    print(f"\nСравнение порогов 0.9 и 0.5:")
    print(f"  Порог 0.9: {t90['count']} объектов")
    print(f"  Порог 0.5: {t50['count']} объектов")

    if t90['count'] > 0 and t50['count'] > 0:
        labels_90 = set(t90['labels'])
        labels_50 = set(t50['labels'])

        classes_90 = [COCO_CATEGORIES[l] for l in labels_90 if l < len(COCO_CATEGORIES)]
        print(f"\n  Классы при пороге 0.9: {classes_90}")

        new_labels = labels_50 - labels_90
        if new_labels:
            new_classes = [COCO_CATEGORIES[l] for l in new_labels if l < len(COCO_CATEGORIES)]
            print(f"  Новые классы при пороге 0.5: {new_classes}")

            # Объекты с низкой уверенностью
            low_conf = [(COCO_CATEGORIES[l], s) for l, s in zip(t50['labels'], t50['scores'])
                       if l in new_labels and s < 0.7 and l < len(COCO_CATEGORIES)]
            if low_conf:
                print(f"  Объекты с низкой уверенностью (<0.7):")
                for name, score in low_conf:
                    print(f"    {name}: {score:.3f}")

**Отчёт по исследованию влияния порога уверенности на результаты детектирования**

1. Количественный анализ

Изображение 1:

Порог 0.9: 3 объекта

Порог 0.7: 4 объекта (+33%)

Порог 0.5: 7 объектов (+133% от порога 0.9)

Порог 0.3: 7 объекта (стабилизация)

Изображение 2:

Порог 0.9: 11 объектов

Порог 0.7: 19 объектов (+73%)

Порог 0.5: 25 объектов (+127% от порога 0.9)

Порог 0.3: 36 объектов (+227% от порога 0.9)

Изображение 3:

Порог 0.9: 6 объектов

Порог 0.7: 12 объектов (+100%)

Порог 0.5: 17 объектов (+183% от порога 0.9)

Порог 0.3: 29 объектов (+383% от порога 0.9)

2. Качественный анализ по изображениям

Изображение 1 (с кошками):

Обнаружено при пороге 0.9: dog, clock
Новые объекты при пороге 0.5: mouse, tv
Объекты с низкой уверенностью (<0.7):

tv: 0.541

tv: 0.540

mouse: 0.538

Наблюдения: При снижении порога с 0.9 до 0.5 появились детекции объектов с уверенностью около 0.54. Вероятно, это ложные срабатывания, так как на изображении с кошками вряд ли присутствуют телевизоры и мыши.

Изображение 2 (уличная сцена):

При пороге 0.5 обнаружено 6 объектов с уверенностью <0.7. Это указывает на значительное количество детекций с низкой достоверностью при использовании низкого порога.

Изображение 3 (уличная сцена 2):

При пороге 0.5 обнаружено 5 объектов с уверенностью <0.7. Наибольший относительный прирост объектов при снижении порога (+183%).

3. Основные закономерности

Экспоненциальный рост количества детекций: При снижении порога от 0.9 до 0.3 количество обнаруженных объектов увеличивается в 2-4 раза.
Насыщение детекций: Для Изображения 1 количество объектов стабилизировалось на пороге 0.5, что может указывать на ограниченное количество реальных объектов на сцене.
Пороговый эффект: Наиболее значительный прирост детекций происходит при снижении порога от 0.7 до 0.5.
Процент низкодостоверных детекций:

Изображение 1: 3 из 7 объектов (43%) имеют уверенность <0.7

Изображение 2: 6 из 25 объектов (24%) имеют уверенность <0.7

Изображение 3: 5 из 17 объектов (29%) имеют уверенность <0.7
4. Выводы о влиянии порога уверенности

При высоком пороге (0.9):

Обнаруживаются только наиболее уверенные объекты
Минимальное количество ложных срабатываний
Пропускается значительная часть объектов (особенно в сложных сценах)
Для Изображения 3 пропускается 65% объектов, которые детектируются при пороге 0.5
При низком пороге (0.5):

Значительно увеличивается полнота детекции (+127-183%)
Появляется много детекций с низкой уверенностью (24-43% от общего количества)
Высока вероятность ложных срабатываний
Визуализация становится перегруженной
5. Рекомендации

Для критически важных систем (безопасность, медицина): использовать порог 0.8-0.9
Для общего применения: оптимальный порог 0.7 (баланс точности и полноты)
Для максимального покрытия: порог 0.5 с обязательной постобработкой (NMS)
Для исследовательских целей: порог 0.3-0.5 для выявления всех возможных детекций
6. Общий вывод

Выбор порога уверенности представляет собой классический компромисс между точностью и полнотой детекции. Более высокие пороги обеспечивают высокую точность за счет пропуска объектов, в то время как низкие пороги максимизируют полноту за счет увеличения ложных срабатываний. Для большинства практических задач рекомендуется порог 0.7 как оптимальный баланс.

3

In [ ]:
print("Загрузка моделей сегментации...")

# Семантическая сегментация (DeepLabv3)
semantic_model = deeplabv3_resnet101(weights=DeepLabV3_ResNet101_Weights.DEFAULT)
semantic_model = semantic_model.to(DEVICE)
semantic_model.eval()
print("DeepLabv3 загружен")

# Поэкземплярная сегментация (Mask R-CNN)
instance_model = maskrcnn_resnet50_fpn(weights=MaskRCNN_ResNet50_FPN_Weights.DEFAULT)
instance_model = instance_model.to(DEVICE)
instance_model.eval()
print("Mask R-CNN загружен")


def get_color_palette(num_classes: int) -> np.ndarray:
    """Создаёт цветовую палитру для визуализации сегментации."""
    np.random.seed(42)
    palette = np.random.randint(0, 255, size=(num_classes, 3))
    palette[0] = [0, 0, 0]  # Фон - чёрный
    return palette


color_palette = get_color_palette(len(COCO_CATEGORIES))

In [ ]:
def preprocess_for_segmentation(image: Image.Image, size: int = 520) -> torch.Tensor:
    """Подготавливает изображение для моделей сегментации."""
    transform = T.Compose([
        T.Resize(size),
        T.ToTensor(),
        T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    return transform(image).unsqueeze(0).to(DEVICE)


# Загрузка тестовых изображений для сегментации
SEGMENTATION_URLS = [
    "http://images.cocodataset.org/val2017/000000397133.jpg",
    "http://images.cocodataset.org/val2017/000000037777.jpg",
]

print("Загрузка изображений для сегментации...")
test_images = []
for url in SEGMENTATION_URLS:
    img = load_image_from_url(url)
    if img is not None:
        test_images.append(img)
        print(f"Загружено: {img.size}")

# Применение моделей
print("\nПрименение семантической сегментации...")
semantic_results = []
for img in test_images:
    input_tensor = preprocess_for_segmentation(img)
    with torch.no_grad():
        output = semantic_model(input_tensor)['out'][0]
        semantic_results.append(output.cpu().numpy())

print("Применение поэкземплярной сегментации...")
instance_results = []
for img in test_images:
    input_tensor = preprocess_for_segmentation(img, size=800)
    with torch.no_grad():
        output = instance_model(input_tensor)[0]
        instance_results.append(output)

print("Сегментация выполнена!")

**3(b) Результаты применения моделей**

Семантическая сегментация (DeepLabv3):

- Модель успешно применена к тестовым изображениям
- Получены карты классов (masks) размерностью [C, H, W], где C - количество классов
- Для каждого пикселя определён наиболее вероятный класс

Поэкземплярная сегментация (Mask R-CNN):

Модель применена к тем же изображениям
 - Получены наборы масок для каждого обнаруженного объекта
 - Каждому объекту соответствует bounding box, класс и маска сегментации
 - Уверенность детекции варьируется в зависимости от объекта

In [ ]:
def compare_segmentation_methods(image_idx: int) -> Tuple[set, dict]:
    """Сравнивает семантическую и поэкземплярную сегментацию.
    
    Args:
        image_idx: Индекс изображения
        
    Returns:
        Кортеж (классы семантической сегм., словарь классов instance сегм.)
    """
    img = test_images[image_idx]
    sem_result = semantic_results[image_idx]
    inst_result = instance_results[image_idx]

    fig, axes = plt.subplots(2, 2, figsize=(16, 12))

    # Оригинал
    axes[0, 0].imshow(img)
    axes[0, 0].set_title("Исходное изображение", fontsize=12, fontweight='bold')
    axes[0, 0].axis('off')

    # Семантическая сегментация
    predictions = sem_result.argmax(0)
    colored_mask = np.zeros((*predictions.shape, 3), dtype=np.uint8)
    for class_idx in range(len(COCO_CATEGORIES)):
        mask = predictions == class_idx
        if mask.any():
            colored_mask[mask] = color_palette[class_idx]

    colored_img = Image.fromarray(colored_mask).resize(img.size, Image.NEAREST)
    sem_overlay = Image.blend(img.convert('RGBA'), colored_img.convert('RGBA'), alpha=0.5)
    axes[0, 1].imshow(sem_overlay)
    axes[0, 1].set_title("Семантическая сегментация (DeepLabv3)", fontsize=12, fontweight='bold')
    axes[0, 1].axis('off')

    # Instance сегментация
    keep = inst_result['scores'] > 0.5
    masks = inst_result['masks'][keep].cpu().numpy()
    labels = inst_result['labels'][keep].cpu().numpy()

    instance_overlay = np.array(img).astype(np.float32) / 255.0
    instance_cmap = plt.cm.get_cmap('tab20', len(masks) + 1)

    for i, (mask, label) in enumerate(zip(masks, labels)):
        if label >= len(COCO_CATEGORIES):
            continue

        mask_resized = (mask[0] > 0.5).astype(np.uint8)
        mask_resized = np.array(Image.fromarray(mask_resized * 255).resize(img.size, Image.NEAREST)) / 255.0

        color = np.array(instance_cmap(i))[:3]
        mask_3d = np.stack([mask_resized] * 3, axis=2)
        instance_overlay = np.where(mask_3d > 0.5, 0.3 * instance_overlay + 0.7 * color, instance_overlay)

    axes[1, 0].imshow(instance_overlay)
    axes[1, 0].set_title("Поэкземплярная сегментация (Mask R-CNN)", fontsize=12, fontweight='bold')
    axes[1, 0].axis('off')

    # Сравнение
    comparison = np.zeros((img.size[1], img.size[0] * 2, 3))
    comparison[:, :img.size[0]] = np.array(sem_overlay)[:, :, :3] / 255.0
    comparison[:, img.size[0]:] = instance_overlay

    axes[1, 1].imshow(comparison)
    axes[1, 1].set_title("Сравнение: слева — семантическая, справа — поэкземплярная", fontsize=12)
    axes[1, 1].axis('off')

    plt.suptitle(f"Сравнение методов сегментации (Изображение {image_idx+1})", fontsize=16, y=1.02)
    plt.tight_layout()
    plt.show()

    # Статистика
    sem_classes = set()
    for class_idx in np.unique(predictions):
        if class_idx < len(COCO_CATEGORIES):
            sem_classes.add(COCO_CATEGORIES[class_idx])

    inst_classes = {}
    for label in labels:
        if label < len(COCO_CATEGORIES):
            name = COCO_CATEGORIES[label]
            inst_classes[name] = inst_classes.get(name, 0) + 1

    return sem_classes, inst_classes


# Сравнительный анализ
print("\nСравнительный анализ методов сегментации:")
print("=" * 60)

for i in range(len(test_images)):
    sem_classes, inst_classes = compare_segmentation_methods(i)

    print(f"\nИзображение {i+1}:")
    print(f"  Семантическая сегментация:")
    for cls in sorted(sem_classes):
        print(f"    - {cls}")

    print(f"  Поэкземплярная сегментация:")
    for cls, count in sorted(inst_classes.items()):
        print(f"    - {cls}: {count} экземпляров")

    common = set(sem_classes) & set(inst_classes.keys())
    print(f"  Общих классов: {len(common)}")

3(d) Сравнение семантической и поэкземплярной сегментации

Результаты сравнения методов

Изображение 1

Семантическая сегментация обнаружила: background, bird, cat, fire hydrant
Поэкземплярная сегментация обнаружила: banana (1 экземпляр), broccoli (1), keyboard (1), person (1)
Общих классов: 0
Изображение 2

Семантическая сегментация обнаружила: background, boat, fire hydrant, horse
Поэкземплярная сегментация обнаружила: traffic light (1 экземпляр)
Общих классов: 0
Отличия семантической и поэкземплярной сегментации

Семантическая сегментация

Присваивает класс каждому пикселю изображения
Не различает отдельные экземпляры объектов одного класса
Все объекты одного класса окрашиваются одинаковым цветом
Объекты одного класса сливаются в единую область
Подходит для задач, где важно знать что находится на изображении, но не сколько
Поэкземплярная сегментация

Выделяет каждый объект как отдельную сущность
Каждый экземпляр получает уникальную маску и bounding box
Разные экземпляры одного класса окрашиваются разными цветами
Позволяет подсчитывать количество объектов
Подходит для задач, где важно знать сколько и где именно находятся объекты
Как семантическая сегментация отображает объекты

Когда в сцене присутствует несколько объектов одного класса, семантическая сегментация объединяет их в единую область. Все пиксели, принадлежащие одному классу, окрашиваются одним цветом без разделения на отдельных животных. Границы между экземплярами размываются или игнорируются.

Как instance-сегментация выделяет каждый экземпляр отдельно

Instance-сегментация сначала детектирует каждый объект отдельно, а затем создает для него уникальную маску. Даже если объекты одного класса перекрываются, каждый из них получает свою собственную маску и идентификатор. Это позволяет точно определять границы каждого экземпляра и отслеживать их по отдельности.

Почему для поэкземплярной сегментации требуется сочетание методов детектирования и сегментации

Детектирование необходимо для идентификации отдельных объектов и определения их приблизительного местоположения
Сегментация требуется для точного выделения границ каждого объекта на пиксельном уровне
Комбинация этих методов позволяет сначала найти объекты, а затем точно выделить их контуры
Без детектирования невозможно было бы различать отдельные экземпляры объектов одного класса
Выводы по эксперименту

Качественные различия моделей: Разные модели обнаружили совершенно разные классы объектов, что указывает на разную специализацию и обучение моделей
Практическая применимость: Выбор метода зависит от задачи. Для анализа сцен подходит семантическая сегментация, для подсчета объектов необходима поэкземплярная
Точность моделей: Результаты показывают, что обе модели имеют ограничения и могут давать разные результаты на одних и тех же изображениях
Визуальное представление: Instance-сегментация предоставляет более информативную визуализацию, показывая не только классы объектов, но и их количество и расположение
Рекомендации: Для сложных задач, требующих работы с отдельными объектами, следует использовать поэкземплярную сегментацию несмотря на ее большую вычислительную сложность